In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
os.listdir('cat_in_dat/')

['train_cat_kaggle.csv', 'test_cat_kaggle.csv', 'y_test_cat_kaggle.csv']

In [3]:
train = pd.read_csv('cat_in_dat/train_cat_kaggle.csv')
test = pd.read_csv('cat_in_dat/test_cat_kaggle.csv')
y_test = pd.read_csv('cat_in_dat/y_test_cat_kaggle.csv', header=None).values.flatten()
y_train = train['target'].values.flatten()
train.drop(['target'], axis=1, inplace=True)
ntrain = train.shape[0]

In [4]:
train_test = pd.concat([train, test])
train_test.drop('id', axis=1, inplace=True)

In [5]:
train.shape

(479549, 24)

In [6]:
y_test.shape

(120451,)

In [7]:
test.shape

(120451, 24)

In [8]:
train_test.shape

(600000, 23)

In [9]:
train_test.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
3,0.0,NaN,1.0,T,N,Red,Triangle,Lion,China,Bassoon,a2e1bf0b1,ae6737c29,8c30b9b0b,690411ac0,05afc0f8b,2.0,Expert,Hot,b,Q,wa,3.0,4.0
4,0.0,0.0,0.0,F,N,Red,Triangle,Hamster,Costa Rica,Bassoon,87a5be0d7,cdc35bd00,1cba571fa,b8e63cace,4d3766412,1.0,Grandmaster,Cold,c,R,rg,5.0,6.0


In [10]:
bin_col = [col for col in train_test.columns if col.startswith('bin_')]
nom_col = [col for col in train_test.columns if col.startswith('nom_')]
ord_col = [col for col in train_test.columns if col.startswith('ord_')]

In [11]:
for col in nom_col + bin_col + ord_col + ['day'] + ['month']:
    fill_value = -1
    if train_test[col].dtype == 'O':
        fill_value = 'missing'
    si = SimpleImputer(strategy='constant', fill_value=fill_value)
    tr = LabelEncoder()
    train_test[col] = si.fit_transform(train_test[col].values.reshape(-1, 1))
    train_test[col] = tr.fit_transform(train_test[col].values)

In [12]:
train_test.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,1,1,1,0,0,2,4,3,5,0,1059,1013,87,1,27,3,0,3,2,20,56,6,3
1,2,2,1,0,1,2,3,0,6,3,210,359,27,69,2112,3,2,5,4,23,151,7,7
2,1,0,1,1,0,2,5,3,2,4,556,361,182,222,1747,3,2,1,7,2,50,5,12
3,1,0,2,1,0,2,5,4,1,0,766,1058,138,93,59,2,1,3,1,16,181,3,4
4,1,1,1,0,0,2,5,3,2,0,645,1221,25,167,692,1,2,1,2,17,159,5,6


In [13]:
train = train_test.iloc[:ntrain]
test = train_test.iloc[ntrain:]

In [14]:
from sklearn.model_selection import ParameterSampler
params = {
    'objective': 'binary:logistic',
    'n_estimators': 10000,
    'n_jobs': -1,
    'verbosity': 1,
    'patience': 20,
    'random_state': 0,
    'tree_method': 'gpu_hist'
}

grid_params = {
    'learning_rate': [.01, .05, .1],
    'max_depth': [3, 5, 7, 9, 12, 15],
    'min_child_weight': [1, 3, 5],
    'subsample': [.6, .7, .8, .9],
    'colsample_bytree': [.6, .7, .8, .9, 1.0],
    'reg_lambda': [.01, .025, .05, .075, .1, .5],
    'reg_alpha': [0., .01, .025, .05, .1, .5],
    'gamma': [.05, .075, .1, .3, .5, .7, 1.]
}

list_params = list(ParameterSampler(grid_params,
                                    n_iter=20,
                                    random_state=0))

for param in list_params:
    param.update(params)

In [15]:
from sklearn.metrics import roc_auc_score
aucs = []
for param in list_params:
    print(param)
    clf = XGBClassifier(**param)
    clf.fit(train.values, y_train,
        eval_set=[(train.values, y_train), (test.values, y_test)],
        eval_metric='auc',
        early_stopping_rounds=params['patience'],
        verbose=5)
    prediction = clf.predict_proba(test.values)
    auc = roc_auc_score(y_test, prediction[: ,1])
    aucs.append(auc)

{'subsample': 0.6, 'reg_lambda': 0.1, 'reg_alpha': 0.0, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.01, 'gamma': 0.075, 'colsample_bytree': 0.8, 'objective': 'binary:logistic', 'n_estimators': 10000, 'n_jobs': -1, 'verbosity': 1, 'patience': 20, 'random_state': 0, 'tree_method': 'gpu_hist'}
[0]	validation_0-auc:0.711577	validation_1-auc:0.699932
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[5]	validation_0-auc:0.735427	validation_1-auc:0.721447
[10]	validation_0-auc:0.744115	validation_1-auc:0.728033
[15]	validation_0-auc:0.746186	validation_1-auc:0.729546
[20]	validation_0-auc:0.748195	validation_1-auc:0.730994
[25]	validation_0-auc:0.749546	validation_1-auc:0.731836
[30]	validation_0-auc:0.75028	validation_1-auc:0.732062
[35]	validation_0-auc:0.751315	validation_1-auc:0.732813
[40]	validation_0-auc:0.75285	validation_1-auc:0.73396
[45]	validation_0-auc:0.75359

[675]	validation_0-auc:0.81754	validation_1-auc:0.758018
[680]	validation_0-auc:0.817977	validation_1-auc:0.758123
[685]	validation_0-auc:0.818413	validation_1-auc:0.758216
[690]	validation_0-auc:0.818791	validation_1-auc:0.75828
[695]	validation_0-auc:0.819248	validation_1-auc:0.758411
[700]	validation_0-auc:0.819787	validation_1-auc:0.758641
[705]	validation_0-auc:0.82019	validation_1-auc:0.758769
[710]	validation_0-auc:0.820591	validation_1-auc:0.758835
[715]	validation_0-auc:0.820958	validation_1-auc:0.758898
[720]	validation_0-auc:0.821298	validation_1-auc:0.758946
[725]	validation_0-auc:0.821691	validation_1-auc:0.759037
[730]	validation_0-auc:0.822036	validation_1-auc:0.759114
[735]	validation_0-auc:0.82251	validation_1-auc:0.759268
[740]	validation_0-auc:0.822896	validation_1-auc:0.759339
[745]	validation_0-auc:0.8234	validation_1-auc:0.759525
[750]	validation_0-auc:0.823858	validation_1-auc:0.759653
[755]	validation_0-auc:0.824195	validation_1-auc:0.759727
[760]	validation_0-a

[1380]	validation_0-auc:0.862956	validation_1-auc:0.767834
[1385]	validation_0-auc:0.863202	validation_1-auc:0.767884
[1390]	validation_0-auc:0.863434	validation_1-auc:0.767897
[1395]	validation_0-auc:0.863675	validation_1-auc:0.767924
[1400]	validation_0-auc:0.863907	validation_1-auc:0.76798
[1405]	validation_0-auc:0.864161	validation_1-auc:0.768041
[1410]	validation_0-auc:0.864411	validation_1-auc:0.768063
[1415]	validation_0-auc:0.864674	validation_1-auc:0.76809
[1420]	validation_0-auc:0.864917	validation_1-auc:0.768119
[1425]	validation_0-auc:0.865195	validation_1-auc:0.768138
[1430]	validation_0-auc:0.865437	validation_1-auc:0.768153
[1435]	validation_0-auc:0.86568	validation_1-auc:0.768173
[1440]	validation_0-auc:0.865923	validation_1-auc:0.768206
[1445]	validation_0-auc:0.866169	validation_1-auc:0.768262
[1450]	validation_0-auc:0.866444	validation_1-auc:0.768265
[1455]	validation_0-auc:0.8667	validation_1-auc:0.768281
[1460]	validation_0-auc:0.866961	validation_1-auc:0.76832
[14

[2080]	validation_0-auc:0.893301	validation_1-auc:0.770718
[2085]	validation_0-auc:0.89349	validation_1-auc:0.770731
[2090]	validation_0-auc:0.893667	validation_1-auc:0.770747
[2095]	validation_0-auc:0.893856	validation_1-auc:0.770747
[2100]	validation_0-auc:0.894039	validation_1-auc:0.770765
[2105]	validation_0-auc:0.894259	validation_1-auc:0.770777
[2110]	validation_0-auc:0.894425	validation_1-auc:0.770774
[2115]	validation_0-auc:0.894619	validation_1-auc:0.77078
[2120]	validation_0-auc:0.894771	validation_1-auc:0.770792
[2125]	validation_0-auc:0.89496	validation_1-auc:0.770813
[2130]	validation_0-auc:0.895144	validation_1-auc:0.770814
[2135]	validation_0-auc:0.895354	validation_1-auc:0.770836
[2140]	validation_0-auc:0.895546	validation_1-auc:0.770841
[2145]	validation_0-auc:0.895746	validation_1-auc:0.77086
[2150]	validation_0-auc:0.895927	validation_1-auc:0.770866
[2155]	validation_0-auc:0.896138	validation_1-auc:0.770876
[2160]	validation_0-auc:0.896315	validation_1-auc:0.77088
[2

[55]	validation_0-auc:0.749902	validation_1-auc:0.747117
[60]	validation_0-auc:0.751504	validation_1-auc:0.74826
[65]	validation_0-auc:0.752927	validation_1-auc:0.749356
[70]	validation_0-auc:0.754416	validation_1-auc:0.75047
[75]	validation_0-auc:0.755827	validation_1-auc:0.75154
[80]	validation_0-auc:0.75701	validation_1-auc:0.752455
[85]	validation_0-auc:0.758685	validation_1-auc:0.753713
[90]	validation_0-auc:0.759841	validation_1-auc:0.754502
[95]	validation_0-auc:0.760657	validation_1-auc:0.755098
[100]	validation_0-auc:0.761541	validation_1-auc:0.755634
[105]	validation_0-auc:0.76254	validation_1-auc:0.756271
[110]	validation_0-auc:0.763709	validation_1-auc:0.757147
[115]	validation_0-auc:0.764484	validation_1-auc:0.757697
[120]	validation_0-auc:0.765557	validation_1-auc:0.758422
[125]	validation_0-auc:0.766183	validation_1-auc:0.758758
[130]	validation_0-auc:0.766919	validation_1-auc:0.759188
[135]	validation_0-auc:0.767864	validation_1-auc:0.759906
[140]	validation_0-auc:0.768

...

[390]	validation_0-auc:0.775051	validation_1-auc:0.764669
[395]	validation_0-auc:0.775307	validation_1-auc:0.76481
[400]	validation_0-auc:0.775604	validation_1-auc:0.765014
[405]	validation_0-auc:0.775833	validation_1-auc:0.76514
[410]	validation_0-auc:0.776025	validation_1-auc:0.765215
[415]	validation_0-auc:0.776309	validation_1-auc:0.765365
[420]	validation_0-auc:0.776651	validation_1-auc:0.76556
[425]	validation_0-auc:0.776852	validation_1-auc:0.765675
[430]	validation_0-auc:0.777163	validation_1-auc:0.765878
[435]	validation_0-auc:0.777398	validation_1-auc:0.765982
[440]	validation_0-auc:0.777635	validation_1-auc:0.766109
[445]	validation_0-auc:0.777962	validation_1-auc:0.766321
[450]	validation_0-auc:0.77815	validation_1-auc:0.766371
[455]	validation_0-auc:0.778361	validation_1-auc:0.766434
[460]	validation_0-auc:0.778563	validation_1-auc:0.766533
[465]	validation_0-auc:0.778818	validation_1-auc:0.766699
[470]	validation_0-auc:0.779202	validation_1-auc:0.766951
[475]	validation_0

[1100]	validation_0-auc:0.800483	validation_1-auc:0.774953
[1105]	validation_0-auc:0.800602	validation_1-auc:0.77496
[1110]	validation_0-auc:0.800765	validation_1-auc:0.77503
[1115]	validation_0-auc:0.800866	validation_1-auc:0.775044
[1120]	validation_0-auc:0.800986	validation_1-auc:0.775066
[1125]	validation_0-auc:0.801116	validation_1-auc:0.775125
[1130]	validation_0-auc:0.801252	validation_1-auc:0.775167
[1135]	validation_0-auc:0.801372	validation_1-auc:0.77519
[1140]	validation_0-auc:0.801504	validation_1-auc:0.775244
[1145]	validation_0-auc:0.801614	validation_1-auc:0.775245
[1150]	validation_0-auc:0.801724	validation_1-auc:0.775246
[1155]	validation_0-auc:0.801832	validation_1-auc:0.77525
[1160]	validation_0-auc:0.801959	validation_1-auc:0.775286
[1165]	validation_0-auc:0.802068	validation_1-auc:0.775313
[1170]	validation_0-auc:0.802175	validation_1-auc:0.775326
[1175]	validation_0-auc:0.80229	validation_1-auc:0.775328
[1180]	validation_0-auc:0.802422	validation_1-auc:0.775342
[1

[285]	validation_0-auc:0.764689	validation_1-auc:0.761891
[290]	validation_0-auc:0.765027	validation_1-auc:0.762225
[295]	validation_0-auc:0.765201	validation_1-auc:0.76239
[300]	validation_0-auc:0.765347	validation_1-auc:0.762481
[305]	validation_0-auc:0.765527	validation_1-auc:0.762621
[310]	validation_0-auc:0.765843	validation_1-auc:0.762883
[315]	validation_0-auc:0.76605	validation_1-auc:0.763034
[320]	validation_0-auc:0.766204	validation_1-auc:0.763153
[325]	validation_0-auc:0.766434	validation_1-auc:0.763349
[330]	validation_0-auc:0.766553	validation_1-auc:0.76342
[335]	validation_0-auc:0.766806	validation_1-auc:0.763629
[340]	validation_0-auc:0.767172	validation_1-auc:0.763936
[345]	validation_0-auc:0.767415	validation_1-auc:0.764144
[350]	validation_0-auc:0.76751	validation_1-auc:0.764227
[355]	validation_0-auc:0.767663	validation_1-auc:0.764348
[360]	validation_0-auc:0.767881	validation_1-auc:0.764524
[365]	validation_0-auc:0.768083	validation_1-auc:0.764654
[370]	validation_0

[995]	validation_0-auc:0.783403	validation_1-auc:0.775362
[1000]	validation_0-auc:0.783466	validation_1-auc:0.775404
[1005]	validation_0-auc:0.783511	validation_1-auc:0.775417
[1010]	validation_0-auc:0.783578	validation_1-auc:0.775437
[1015]	validation_0-auc:0.78366	validation_1-auc:0.775469
[1020]	validation_0-auc:0.783719	validation_1-auc:0.775503
[1025]	validation_0-auc:0.783766	validation_1-auc:0.775509
[1030]	validation_0-auc:0.78382	validation_1-auc:0.775534
[1035]	validation_0-auc:0.783886	validation_1-auc:0.775555
[1040]	validation_0-auc:0.783951	validation_1-auc:0.775578
[1045]	validation_0-auc:0.784032	validation_1-auc:0.775633
[1050]	validation_0-auc:0.784143	validation_1-auc:0.775711
[1055]	validation_0-auc:0.784188	validation_1-auc:0.775717
[1060]	validation_0-auc:0.784256	validation_1-auc:0.775733
[1065]	validation_0-auc:0.784308	validation_1-auc:0.775748
[1070]	validation_0-auc:0.784387	validation_1-auc:0.775796
[1075]	validation_0-auc:0.784449	validation_1-auc:0.775835


[340]	validation_0-auc:0.804694	validation_1-auc:0.766232
[345]	validation_0-auc:0.805312	validation_1-auc:0.766418
[350]	validation_0-auc:0.805912	validation_1-auc:0.766585
[355]	validation_0-auc:0.806453	validation_1-auc:0.766672
[360]	validation_0-auc:0.807038	validation_1-auc:0.766799
[365]	validation_0-auc:0.807581	validation_1-auc:0.766893
[370]	validation_0-auc:0.808102	validation_1-auc:0.767003
[375]	validation_0-auc:0.808819	validation_1-auc:0.767328
[380]	validation_0-auc:0.809435	validation_1-auc:0.767485
[385]	validation_0-auc:0.80992	validation_1-auc:0.767566
[390]	validation_0-auc:0.810462	validation_1-auc:0.76769
[395]	validation_0-auc:0.811116	validation_1-auc:0.767948
[400]	validation_0-auc:0.811599	validation_1-auc:0.768061
[405]	validation_0-auc:0.812081	validation_1-auc:0.768167
[410]	validation_0-auc:0.812557	validation_1-auc:0.768224
[415]	validation_0-auc:0.813099	validation_1-auc:0.768323
[420]	validation_0-auc:0.813618	validation_1-auc:0.768439
[425]	validation

In [17]:
for i,j in zip(aucs, list_params):
    print(i)
    print(j)

0.7710714465782478
{'subsample': 0.6, 'reg_lambda': 0.1, 'reg_alpha': 0.0, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.01, 'gamma': 0.075, 'colsample_bytree': 0.8, 'objective': 'binary:logistic', 'n_estimators': 10000, 'n_jobs': -1, 'verbosity': 1, 'patience': 20, 'random_state': 0, 'tree_method': 'gpu_hist'}
0.7604324410815388
{'subsample': 0.9, 'reg_lambda': 0.05, 'reg_alpha': 0.1, 'min_child_weight': 3, 'max_depth': 12, 'learning_rate': 0.05, 'gamma': 0.7, 'colsample_bytree': 0.8, 'objective': 'binary:logistic', 'n_estimators': 10000, 'n_jobs': -1, 'verbosity': 1, 'patience': 20, 'random_state': 0, 'tree_method': 'gpu_hist'}
0.7746518003293391
{'subsample': 0.9, 'reg_lambda': 0.05, 'reg_alpha': 0.01, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.075, 'colsample_bytree': 0.8, 'objective': 'binary:logistic', 'n_estimators': 10000, 'n_jobs': -1, 'verbosity': 1, 'patience': 20, 'random_state': 0, 'tree_method': 'gpu_hist'}
0.7755512505826565
{'sub